Try and send some data to bigquery to check schemas and whatnot

In [52]:
from google.cloud import bigquery
import os
import pandas as pd
from datetime import datetime
import json


In [2]:
# Set environment variable
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "/home/ed/code/Eatkin/gcp/reddit-sentiment-400608-24bc7c65b16e.json"


In [15]:
comments_df = pd.read_csv('../reddit_data/reddit_comments_2023-10-03.csv', index_col=0)
posts_df = pd.read_csv('../reddit_data/reddit_posts_2023-10-03.csv', index_col=0)


In [16]:
# Add a date column to the dataframe
comments_df['date'] = datetime.now().strftime('%Y-%m-%d')
posts_df['date'] = datetime.now().strftime('%Y-%m-%d')


In [5]:
project_id = "reddit-sentiment-400608"
dataset_id = "wallstreetbets"
table_id = "reddit_comments"


In [6]:
# Establish connection
client = bigquery.Client(project=project_id)


In [50]:
# Set up schema
comments_schema = [
    bigquery.SchemaField("date", "DATE"),
    bigquery.SchemaField("text", "STRING"),
    bigquery.SchemaField("score", "INTEGER"),
    bigquery.SchemaField("level", "INTEGER"),
    bigquery.SchemaField("post", "STRING"),
    bigquery.SchemaField("url", "STRING"),
    bigquery.SchemaField("sentiment", "FLOAT"),
    bigquery.SchemaField("joy", "FLOAT"),
    bigquery.SchemaField("optimism", "FLOAT"),
    bigquery.SchemaField("anger", "FLOAT"),
    bigquery.SchemaField("sadness", "FLOAT"),
]

posts_schema = [
    bigquery.SchemaField("date", "DATE"),
    bigquery.SchemaField("ids", "STRING"),
    bigquery.SchemaField("titles", "STRING"),
    bigquery.SchemaField("scores", "INTEGER"),
    bigquery.SchemaField("controversiality", "FLOAT"),
    bigquery.SchemaField("bodies", "STRING"),
    bigquery.SchemaField("original", "BOOLEAN"),
    bigquery.SchemaField("num_comments", "INTEGER"),
    bigquery.SchemaField("url", "STRING"),
    bigquery.SchemaField("sentiment", "FLOAT"),
    bigquery.SchemaField("joy", "FLOAT"),
    bigquery.SchemaField("optimism", "FLOAT"),
    bigquery.SchemaField("anger", "FLOAT"),
    bigquery.SchemaField("sadness", "FLOAT"),
]


In [62]:
# Set up job configs
comments_job_config = bigquery.LoadJobConfig(
    schema=comments_schema,
    write_disposition=bigquery.WriteDisposition.WRITE_APPEND,
)
posts_job_config = bigquery.LoadJobConfig(
    schema=posts_schema,
    write_disposition=bigquery.WriteDisposition.WRITE_APPEND,
)

# Bigquery requires dataframe to be in a list so convert to records format
comments_records = comments_df.to_dict(orient="records")
posts_records = posts_df.to_dict(orient="records")


In [21]:
# Let's try and push it
print("Pushing comments to Bigquery")
client.load_table_from_json(
    comments_records, f"{project_id}.{dataset_id}.{table_id}", job_config=comments_job_config
).result()


Pushing comments to Bigquery
Pushing posts to Bigquery


BadRequest: 400 Error while reading data, error message: JSON table encountered too many errors, giving up. Rows: 1; errors: 1. Please look into the errors[] collection for more details.

In [65]:
# Required because of NaNs
for record in posts_records:
    if type(record['bodies']) == float:
        record['bodies'] = ''


In [66]:
# Switch table id to posts
table_id = "reddit_posts"

print("Pushing posts to Bigquery")
client.load_table_from_json(
    posts_records, f"{project_id}.{dataset_id}.{table_id}", job_config=posts_job_config
).result()


Pushing posts to Bigquery


LoadJob<project=reddit-sentiment-400608, location=US, id=b021f8f5-0216-4ba0-89ef-7a0ba1788008>